## Ensemble Learning

* 여러 개의 Classifier를 생성하고 그 예측을 결합하여 보다 정확한 최종 예측을 도출하는 기법
* 단일 모델의 약점을 다수의 모델을 결합하여 보완
* 뛰어난 성능을 가진 모델들로만 구성하는 것보다 성능이 떨어지더라도 서로 다른 유형의 모델을 섞는 것이 전체 성능이 더 우수할 수 있음
* Ensemble 유형
  * Voting
    * 일반적으로 서로 다른 알고리즘을 가진 Classifier들을 결합하여 같은 Data Set으로 Classifier별 성능을 비교
    * Voting의 유형
      * Hard Voting
        * 다수의 classifier 간 다수결로 최종 class를 결정
      * Soft Voting
        * 다수의 calssifier들의 class 확률을 평균하여 결정
        * Hard Voting보다 예측 성능이 우수함
  * Bagging
    * 각각의 Classifier는 같은 유형의 알고리즘이지만 Sampling Data를 다르게하여 학습을 수행하여 성능을 비교
    * Random Forest
  * Boosting
    * 에이다 부스팅
    * 그래디언트 부스팅
    * XGBoost
    * LightGBM

### Voting Classifier
**위스콘신 유방암 데이터 load**

In [1]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


**VotingClassifier로 LogisticRegression과 KNN을 Voting 방식으로 결합하고 성능 비교**

In [2]:
# 개별 모델은 LogisticRegression과 KNN
lr_clf = LogisticRegression(max_iter=3000)
knn_clf = KNeighborsClassifier(n_neighbors = 8)

# 개별 모델을 Soft Voting 기반의 Ensemble 모델로 구현한 분류기
vo_clf = VotingClassifier( estimators=[('LR',lr_clf),('KNN',knn_clf)] , voting='soft' )

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2 , random_state= 156)

# VotingClassifier 학습/예측/평가
vo_clf.fit(X_train , y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test , pred)))

# 개별 모델의 학습/예측/평가.
classifiers = [lr_clf, knn_clf]
for classifier in classifiers:
    classifier.fit(X_train , y_train)
    pred = classifier.predict(X_test)
    class_name= classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(class_name, accuracy_score(y_test , pred)))

Voting 분류기 정확도: 0.9474
LogisticRegression 정확도: 0.9649
KNeighborsClassifier 정확도: 0.9386
